# Running RAVEN on a multi-basin NetCDF file in parallel

Here we use birdy's WPS client to launch the GR4JCN hydrological model on the server for 2 basins in parallel. The NetCDF is thus a 2d file. We then analyze the outputs. 

In [1]:
#Import necessary packages and start the service 
from birdy import WPSClient

from example_data import TESTDATA
import datetime as dt
from urllib.request import urlretrieve
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt

# url = "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps"
url = "http://localhost:9099/wps"
wps = WPSClient(url)

In [6]:
# The model parameters for gr4jcn for both catchments. Can either be a string of comma separated values, a list, an array or a named tuple. 
gr4jcn1 ='0.529, -3.396, 407.29, 1.072, 16.9, 0.947'
gr4jcn2 ='0.28, -3.6, 380.9, 1.092, 14.6, 0.831'

params=[gr4jcn1,gr4jcn2]

# Forcing files. Raven uses the same forcing files for all and extracts the information it requires for each model.
ts=TESTDATA['input2d']

# Model configuration parameters.
config = dict(
    start_date=[dt.datetime(2000, 1, 1),dt.datetime(2000, 1, 1)],
    end_date=[dt.datetime(2002, 1, 1),dt.datetime(2002, 1, 1)],
    area=[4250.6,5000],
    elevation=[843.0,780],
    latitude=[54.4848,48.0],
    longitude=[-123.3659,-122.99],
    nc_index=[0,0],
    )

# Launch the WPS to get the multi-model results.  Note the "gr4jcn" and "hmets" keys.
resp=wps.raven_gr4j_cemaneige(ts=str(ts),params=params, **config) 

# And get the response
# With `asobj` set to False, only the reference to the output is returned in the response. 
# Setting `asobj` to True will retrieve the actual files and copy the locally.

### asobj= True is failing!
[hydrograph, storage, solution, diagnostics] = resp.get(asobj=True)

AttributeError: 'str' object has no attribute 'identifier'

In [4]:
# Print the diagnostics for both catchments
print(diagnostics)

http://localhost:9099/outputs/1ee9118e-cf52-11e9-b8c4-484d7ef8d8d1/Diagnostics.zip


The `hydrograph` and `storage` outputs are netCDF files storing the time series. These files are opened by default using `xarray`, which provides convenient and powerful time series analysis and plotting tools. 

In [5]:
# Plot the hydrographs for both basins
hydrograph.q_sim.isel(nbasins=0).plot.line(hue='realization')
hydrograph.q_sim.isel(nbasins=1).plot.line(hue='realization')

AttributeError: 'str' object has no attribute 'q_sim'